# Lecture : Graph Convolutional Networks

## Lab 01 : ChebNets -- Solution

### Xavier Bresson, Nian Liu

Defferrard, Bresson, Vandergheynst, Convolutional Neural Networks on Graphs with Fast Localized Spectral Filtering, 2016  
https://arxiv.org/pdf/1606.09375


In [1]:
# For Google Colaboratory
import sys, os
if 'google.colab' in sys.modules:
    # mount google drive
    from google.colab import drive
    drive.mount('/content/gdrive')
    path_to_file = '/content/gdrive/My Drive/CS5284_2024_codes/codes/08_Graph_Convnets'
    print(path_to_file)
    # change current path to the folder containing "path_to_file"
    os.chdir(path_to_file)
    !pwd
    

In [2]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import collections
import time
import numpy as np
import sys
sys.path.insert(0, 'lib/')
%load_ext autoreload
%autoreload 2


## MNIST


In [3]:
# Load small MNIST
[train_data, train_label, test_data, test_label] = torch.load('datasets/MNIST_1k.pt')
print('train_data',train_data.size())
print('train_label',train_label.size())
print('test_data',test_data.size())
print('test_label',test_label.size())


train_data torch.Size([1000, 784])
train_label torch.Size([1000])
test_data torch.Size([100, 784])
test_label torch.Size([100])


## Compute coarsened graphs


In [4]:
from lib.grid_graph import grid_graph
from lib.coarsening import coarsen
from lib.coarsening import lmax_L
from lib.coarsening import perm_data
from lib.coarsening import rescale_L

# Construct grid graph
t_start = time.time()
grid_side = 28  # Each image is 28 * 28
number_edges = 8  # Each pixel has eight neighbors
A = grid_graph(grid_side, number_edges, 'euclidean') # create graph of Euclidean grid

# Compute coarsened graphs
num_coarsening_levels = 4
L, perm = coarsen(A, num_coarsening_levels)

# Compute largest eigenvalue of graph Laplacians
lmax = []
for i in range(num_coarsening_levels):
    lmax.append(lmax_L(L[i]))
print('lmax: ' + str([lmax[i] for i in range(num_coarsening_levels)]))

# Reindex nodes to satisfy a binary tree structure
train_data = perm_data(train_data, perm)
test_data = perm_data(test_data, perm)
train_data = torch.tensor(train_data).float()
test_data = torch.tensor(test_data).float()
print(train_data.size())
print(test_data.size())

print('Execution time: {:.2f}s'.format(time.time() - t_start))
del perm


nb edges:  6396
Heavy Edge Matching coarsening with Xavier version
Layer 0: M_0 = |V| = 944 nodes (160 added), |E| = 3198 edges
Layer 1: M_1 = |V| = 472 nodes (67 added), |E| = 1619 edges
Layer 2: M_2 = |V| = 236 nodes (23 added), |E| = 784 edges
Layer 3: M_3 = |V| = 118 nodes (5 added), |E| = 387 edges
Layer 4: M_4 = |V| = 59 nodes (0 added), |E| = 190 edges
lmax: [1.3857534, 1.3440951, 1.2102374, 1.0000005]
torch.Size([1000, 944])
torch.Size([100, 944])
Execution time: 0.34s


## Question 1: Implement ChebNet based on the CNN LeNet-5 architecture

- First layer : CL with 32 features
- Second layer : MaxPooling to reduce graph size by a factor 4
- Third layer : CL with 64 features
- Fourth layer : MaxPooling to reduce graph size by a factor 4
- Fifth layer : Fully connected (or linear) layer with 512 features 
- Last layer : Fully connected (or linear) layer with 10 output values for 10 classes 


Instructions:

Step 1: Define the model architecture with the constructor `def __init__()`.
  - Use [torch.nn.Linear()](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html#torch.nn.Linear) to linearly transform the feature dimensions. 
  - Convert the SciPy sparse graph Laplacian and its coarsened versions to PyTorch sparse matrices with [torch.sparse.FloatTensor(indices, data, shape)](https://pytorch.org/docs/stable/sparse.html).
  - For a [scipy.sparse.coo_matrix](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.coo_matrix.html) L, `L.row`, `L.col` and `L.data` are the row indices, column indices, and weights for each edge, respectively.

Step 2: Define MaxPooling layers in `def graph_max_pool()` using [torch.nn.MaxPool1d()](https://pytorch.org/docs/stable/generated/torch.nn.MaxPool1d.html#torch.nn.MaxPool1d).
  - Note that the pooling is done along the node dimension (analogous to a 1D sequence) and not the feature dimension.
  - Use [torch.permute()](https://pytorch.org/docs/stable/generated/torch.permute.html#torch.permute) to reorder the tensor dimensions to apply `torch.nn.MaxPool1d()`, and [.contiguous()](https://pytorch.org/docs/stable/generated/torch.Tensor.contiguous.html#torch.Tensor.contiguous) to ensure a contiguous memory layout for the target tensor.

Step 3: Implement ChebNet convolution in `def graph_conv_cheby()`.
  - Compute explicitly the first two Chebyshev terms, then code the recursive formula.
  - Use [torch.sparse.mm()](https://pytorch.org/docs/stable/generated/torch.sparse.mm.html#torch.sparse.mm) to perform matrix multiplication between two sparse matrices, which is less memory consuming than [torch.mm()](https://pytorch.org/docs/stable/generated/torch.mm.html#torch.mm).
  - Node features are integrated into the first two terms, so the following terms automatically contain node features via recursive iterations, see Slides 34 and 35 from the lecture.
  - Combine the results from all intermediate Chebyshev orders to get the output, by linearly transforming them to match the input dimension of the next layer.

Step 4: Implement the forward pass in `def forward()`.
  - For CL1 and CL2, `graph_conv_cheby -> torch.relu (non-linear activation) -> graph_max_pool`
  - For FC1 and FC2, `fc1 -> torch.relu (non-linear activation) -> dropout -> fc2`  
    

In [5]:
# class definition
class ChebNet_LeNet5(nn.Module):
    def __init__(self, net_parameters, Ls, lmax):
        super().__init__()
        # parameters
        # D: dimension of input features
        # CL1_F, CL2_F: dimensions of output from the First and Third layers
        # CL1_K, CL2_K: orders of Chebyshev terms used in CL1 and CL2
        # FC1_F, FC2_F: dimensions of output from the Fifth and Last layers
        # FC1Fin: dimensions of input for the Fifth layer
        D, CL1_F, CL1_K, CL2_F, CL2_K, FC1_F, FC2_F = net_parameters
        FC1Fin = CL2_F*(D//16)
        
        ########################################
        # YOUR CODE STARTS
        # Step 1.1: Using the input and output dims at CL1, CL2, FC1, FC2 layers, define the corresponding feature transformation functions
        ########################################
        # graph CL1
        self.cl1 = nn.Linear(CL1_K*1, CL1_F) # the dim of input feature is 1
        self.CL1_K = CL1_K; self.CL1_F = CL1_F
        
        # graph CL2
        self.cl2 = nn.Linear(CL2_K*CL1_F, CL2_F) 
        self.CL2_K = CL2_K; self.CL2_F = CL2_F
        
        # FC1
        self.fc1 = nn.Linear(FC1Fin, FC1_F) 
        self.FC1Fin = FC1Fin
        
        # FC2
        self.fc2 = nn.Linear(FC1_F, FC2_F)
        ########################################
        # YOUR CODE ENDS
        ########################################

        # Compute the pytorch Laplacian and its coarsened versions
        self.L = []
        for i in range(num_coarsening_levels+1):
            L = Ls[i] 
            # rescale Laplacian: shift L into the definition domain of Chebyshev expansion λ ∈ [-1, 1]
            lmax = lmax_L(L)
            L = rescale_L(L, lmax) 
            # convert scipy sparse matric L to pytorch
            L = L.tocoo()
            ########################################
            # YOUR CODE STARTS
            # Step 1.2: Convert each L into a pytorch sparse tensor
            ########################################
            indices = np.column_stack((L.row, L.col)).T 
            indices = indices.astype(np.int64)
            indices = torch.from_numpy(indices)
            indices = indices.type(torch.LongTensor)
            L_data = L.data.astype(np.float32)
            L_data = torch.from_numpy(L_data) 
            L_data = L_data.type(torch.FloatTensor)
            L = torch.sparse.FloatTensor(indices, L_data, torch.Size(L.shape))
            ########################################
            # YOUR CODE ENDS
            ########################################
            L = Variable( L , requires_grad=False)
            self.L.append(L)
        
    # Max pooling of size p (p must be a power of 2)
    def graph_max_pool(self, x, p): 
        # B, V, F = x.shape
        # B = batch size
        # V = num vertices
        # F = num features
        if p > 1: 
            ########################################
            # YOUR CODE STARTS
            # Step 2: Exchange the vertex dim and feature dim, do pooling along the last dim.
            ########################################
            x = x.permute(0,2,1).contiguous()  # x = B x F x V
            x = nn.MaxPool1d(p)(x)             # B x F x V/p          
            x = x.permute(0,2,1).contiguous()  # x = B x V/p x F
            ########################################
            # YOUR CODE ENDS
            ########################################
            return x  
        else:
            return x   
            
    # Graph convolution layer
    def graph_conv_cheby(self, x, cl, L, Fout, K):
        # parameters
        # B = batch size
        # V = num vertices
        # Fin = num input features
        # Fout = num output features
        # K = Chebyshev order and support size
        B, V, Fin = x.size(); B, V, Fin = int(B), int(V), int(Fin) 
        
        # Transform to Chebyshev basis
        ########################################
        # YOUR CODE STARTS
        # Step 3.1: Compute the first two Chebyshev terms, integrate 'x', concatenate them
        # The first Chebyshev term: T_0(L)=I
        # The second Chebyshev term: T_1(L)=L
        ########################################
        x0 = x.permute(1,2,0).contiguous()  # V x Fin x B
        x0 = x0.view([V, Fin*B])            # V x Fin*B

        # x: Concatenate the outputs from each Chebyshev term
        x = x0.unsqueeze(0)                 # 1 x V x Fin*B
        if K > 1: 
            x1 = torch.sparse.mm(L,x0)             # V x Fin*B
            x = torch.cat((x, x1.unsqueeze(0)),0)  # 2 x V x Fin*B
        ########################################
        # YOUR CODE ENDS
        ########################################
        
        for k in range(2, K):
            ########################################
            # YOUR CODE STARTS
            # Step 3.2: Apply recursive formula, concatenate results, update x0, x1 as x1, x2
            # The following Chebyshev terms: T_2(L)=2*LT_1(L)-T_0(L)
            ########################################
            x2 = 2 * torch.sparse.mm(L,x1) - x0  
            x = torch.cat((x, x2.unsqueeze(0)),0)  # (k+1) x V x Fin*B
            x0, x1 = x1, x2  
            ########################################
            # YOUR CODE ENDS
            ########################################
        x = x.view([K, V, Fin, B])           # K x V x Fin x B     
        x = x.permute(3,1,2,0).contiguous()  # B x V x Fin x K       
        x = x.view([B*V, Fin*K])             # B*V x Fin*K
        ########################################
        # YOUR CODE STARTS
        # Step 3.3: Linear transform Fin features to obtain Fout features
        ########################################
        x = cl(x)                            # B*V x Fout  
        ########################################
        # YOUR CODE ENDS
        ########################################
        x = x.view([B, V, Fout])             # B x V x Fout
        return x

    def forward(self, x):
        # graph CL1
        x = x.unsqueeze(2) # B x V x Fin=1, images in MNIST only have one channel/feature
        x = self.graph_conv_cheby(x, self.cl1, self.L[0], self.CL1_F, self.CL1_K)
        x = torch.relu(x)
        x = self.graph_max_pool(x, 4)
        # graph CL2
        ########################################
        # YOUR CODE STARTS
        # Step 4.1: Implement CL2
        ########################################
        x = self.graph_conv_cheby(x, self.cl2, self.L[2], self.CL2_F, self.CL2_K)
        x = torch.relu(x)
        x = self.graph_max_pool(x, 4)
        ########################################
        # YOUR CODE ENDS
        ########################################
        # FC1
        x = x.view(-1, self.FC1Fin) # resize the tensor
        ########################################
        # YOUR CODE STARTS
        # Step 4.2: Implement FC1
        ########################################
        x = self.fc1(x)
        x = torch.relu(x)
        ########################################
        # YOUR CODE ENDS
        ########################################
        # FC2
        x = self.fc2(x) 
        return x
        
    def update_learning_rate(self, optimizer, lr): # Adjust the learning rate based on the number of epochs
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr
        return optimizer


## Question 2: Print basic information of ChebNet, and test the forward and backward passes with one batch

Instructions:

Step 1: Count the number of model parameters.
  - For a pytorch model `net`, [net.parameters()](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module) returns an iterator over module parameters. [torch.numel()](https://pytorch.org/docs/stable/generated/torch.numel.html#torch.numel) returns the total number of elements in the input tensor.

Step 2: Implement the loss function that includes a standard cross-entropy classfication loss and a L2 regularization loss for the learnable parameters of the network.
  - Use [torch.nn.CrossEntropyLoss()](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss) to compute the cross-entropy loss between input logits and target labels.
  - Use `for param in net.parameters():` to access to the learnable parameters for the L2 regularization loss defined as L$(w_1,w_2)=\sum_k\|w_k\|^2$  


In [6]:
# network parameters
D = train_data.shape[1]
CL1_F = 32
CL1_K = 25
CL2_F = 64
CL2_K = 25
FC1_F = 512
FC2_F = 10
net_parameters = [D, CL1_F, CL1_K, CL2_F, CL2_K, FC1_F, FC2_F]

# instantiate ChebNet
net = ChebNet_LeNet5(net_parameters, L, lmax)
print(net)

def display_num_param(net):
    nb_param = 0
    ########################################
    # YOUR CODE STARTS
    # Step 1: Count the number of model parameters
    ########################################
    for param in net.parameters():
        nb_param += param.numel()
    ########################################
    # YOUR CODE ENDS
    ########################################
    print('Number of parameters: {} ({:.2f} million)'.format(nb_param, nb_param/1e6))
    return nb_param/1e6
display_num_param(net)
    
# extract one batch
batch_size = 10
indices = torch.randperm(train_data.shape[0])
batch_idx = indices[:batch_size]
print('batch_idx: ',batch_idx)
train_x, train_y = train_data[batch_idx,:], train_label[batch_idx]

# Forward 
y = net(train_x)

# backward
def loss_reg(lossCE, net, y, y_target, l2_regularization):
    CE_loss = 0.0
    l2_loss = 0.0
    ########################################
    # YOUR CODE STARTS
    # Step 2: Compute CE_loss and l2_loss
    ########################################
    CE_loss = lossCE(y, y_target)
    for param in net.parameters():
        data = param* param
        l2_loss += data.sum()   
    ########################################
    # YOUR CODE ENDS
    ########################################
    loss = 0.5* l2_regularization* l2_loss + CE_loss
    return loss
    
lossCE = nn.CrossEntropyLoss()
l2_regularization = 1e-3 
loss = loss_reg(lossCE, net, y, train_y, l2_regularization)
loss.backward()

# Update 
learning_rate = 0.05
optimizer = torch.optim.SGD( net.parameters(), lr=learning_rate, momentum=0.9 )
optimizer.zero_grad()
optimizer.step()


ChebNet_LeNet5(
  (cl1): Linear(in_features=25, out_features=32, bias=True)
  (cl2): Linear(in_features=800, out_features=64, bias=True)
  (fc1): Linear(in_features=3776, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=10, bias=True)
)
Number of parameters: 1991050 (1.99 million)
batch_idx:  tensor([946, 145, 856, 427, 730, 484, 168, 637, 341, 521])


/var/folders/44/qs4c3c1d6ms37t9j12bv14240000gn/T/ipykernel_26777/3455975606.py:56: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at /Users/runner/miniforge3/conda-bld/libtorch_1715184444117/work/torch/csrc/utils/tensor_new.cpp:607.)
  L = torch.sparse.FloatTensor(indices, L_data, torch.Size(L.shape))


## Question 3: Training ChebNet

Instructions:

Step 1: Initialize the model, the optimizer, and the loss function.

Step 2: Repeat the training loop `num_epochs` times.
- At the beginnig of a new epoch, shuffle the samples, reset the loss value and accuracy to zero, and set the `net` in training mode: [net.train()](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.train).
- During each epoch, the sequence of instructions `batch data -> model -> output logits -> loss -> backward propagation -> parameter update -> evaluation` is used.
- Gradient of the loss w.r.t. the net paramaters is automatically computed with [loss.backward()](https://pytorch.org/docs/stable/generated/torch.autograd.backward.html#torch.autograd.backward).
- One update step of the parameter values is performed with [optimizer.step()](https://pytorch.org/docs/stable/generated/torch.optim.Optimizer.step.html#torch.optim.Optimizer.step). Do not forget to zero the gradient at each mini-batch with [optimizer.zero_grad()](https://pytorch.org/docs/stable/generated/torch.optim.Optimizer.zero_grad.html#torch.optim.Optimizer.zero_grad).
- After each epoch, update the learning rate, and evaluate the testset accuracy with [torch.no_grad()](https://pytorch.org/docs/stable/generated/torch.no_grad.html#torch.no_grad) to disable gradient calculation, and [net.eval()](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module) to set the `net` in evaluation mode.
      

In [7]:
# network parameters
D = train_data.shape[1]
CL1_F = 32
CL1_K = 25
CL2_F = 64
CL2_K = 25
FC1_F = 512
FC2_F = 10
net_parameters = [D, CL1_F, CL1_K, CL2_F, CL2_K, FC1_F, FC2_F]

# instantiate ChebNet
net = ChebNet_LeNet5(net_parameters, L, lmax)
print(net)
display_num_param(net)

# optimization parameters
lr = 0.05 # learning_rate
init_lr = lr
l2_regularization = 1e-3 
batch_size = 100
num_epochs = 20
num_train_data = train_data.shape[0]
nb_iter = int(num_epochs * num_train_data) // batch_size
print('num_epochs=',num_epochs,', num_train_data=',num_train_data,', nb_iter=',nb_iter)

# Optimizer
########################################
# YOUR CODE STARTS
# Step 1: Initialize loss, optimizer and evaluation
########################################
lossCE = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD( net.parameters(), lr=lr, momentum=0.9 )
########################################
# YOUR CODE ENDS
########################################

def evaluation(y_predicted, y_label):
    _, class_predicted = torch.max(y_predicted, 1)
    return 100.0* (class_predicted == y_label).sum()/ y_predicted.size(0)
    
# loop over epochs
num_data = 0
for epoch in range(num_epochs):  

    # reshuffle 
    indices = torch.randperm(num_train_data)
    
    # reset time
    t_start = time.time()
    
    # extract batches
    running_loss = 0.0
    running_accuray = 0
    running_total = 0
    net.train()
    for idx in range(0,num_train_data,batch_size):

        # extract batches
        train_x, train_y = train_data[idx:idx+batch_size,:], train_label[idx:idx+batch_size]
            
        # Forward 
        y = net(train_x)
        
        # backward
        loss = loss_reg(lossCE, net, y, train_y, l2_regularization)
        ########################################
        # YOUR CODE STARTS
        # Step 2: Apply backward propagation and update net parameters
        ########################################
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        ########################################
        # YOUR CODE ENDS
        ########################################
        
        # Accuracy
        acc_train = evaluation(y.detach(), train_y)
        
        # loss, accuracy
        num_data += batch_size 
        running_loss += loss.detach()
        running_accuray += acc_train
        running_total += 1
      
    # print 
    print('epoch= %d, loss(train)= %.3f, accuracy(train)= %.3f, time= %.3f, lr= %.5f' % 
          (epoch+1, running_loss/running_total, running_accuray/running_total, time.time()-t_start, lr))
 
    # update learning rate 
    lr = init_lr * pow( 0.95 , float(num_data// num_train_data) )
    optimizer = net.update_learning_rate(optimizer, lr)
    
    # Test set
    with torch.no_grad():
        net.eval()
        running_accuray_test = 0
        running_total_test = 0
        num_test_data = test_data.size(0)
        indices_test = torch.arange(num_test_data)
        t_start_test = time.time()
        for idx in range(0,num_test_data,batch_size):
            test_x, test_y = test_data[idx:idx+batch_size,:], test_label[idx:idx+batch_size]
            y = net(test_x)
            acc_test = evaluation(y.detach(), test_y)
            running_accuray_test += acc_test
            running_total_test += 1
        t_stop_test = time.time() - t_start_test
        print('  accuracy(test) = %.3f %%, time= %.3f' % (running_accuray_test / running_total_test, t_stop_test))  



ChebNet_LeNet5(
  (cl1): Linear(in_features=25, out_features=32, bias=True)
  (cl2): Linear(in_features=800, out_features=64, bias=True)
  (fc1): Linear(in_features=3776, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=10, bias=True)
)
Number of parameters: 1991050 (1.99 million)
num_epochs= 20 , num_train_data= 1000 , nb_iter= 200
epoch= 1, loss(train)= 2.402, accuracy(train)= 10.500, time= 5.352, lr= 0.05000
  accuracy(test) = 8.000 %, time= 0.214
epoch= 2, loss(train)= 2.365, accuracy(train)= 18.200, time= 3.826, lr= 0.04750
  accuracy(test) = 34.000 %, time= 0.187
epoch= 3, loss(train)= 2.242, accuracy(train)= 34.900, time= 6.988, lr= 0.04512
  accuracy(test) = 46.000 %, time= 0.270
epoch= 4, loss(train)= 1.778, accuracy(train)= 49.300, time= 7.281, lr= 0.04287
  accuracy(test) = 63.000 %, time= 0.191
epoch= 5, loss(train)= 1.034, accuracy(train)= 71.000, time= 5.536, lr= 0.04073
  accuracy(test) = 74.000 %, time= 0.206
epoch= 6, loss(train)= 0.774, accur